# Árbol de Decisión de Regresión — Dataset Real (Diabetes)

Este cuaderno entrena y evalúa un **árbol de decisión de regresión** usando el dataset de **diabetes** de scikit-learn.

Incluye:
- Exploración y limpieza (capado de outliers por IQR e imputación de faltantes)
- División train/test
- Modelo base y **tuning** con `GridSearchCV`
- **Curva de aprendizaje**, **importancias** y **visualización del árbol** (truncado)

> Nota: Se usa `matplotlib` sin estilos predefinidos y una figura por gráfico.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
import math
np.random.seed(42)


## Funciones auxiliares

In [ ]:
def cap_outliers_iqr(df, cols):
    capped = df.copy()
    for c in cols:
        q1 = capped[c].quantile(0.25)
        q3 = capped[c].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        capped[c] = np.clip(capped[c], lower, upper)
    return capped

def report_metrics(y_true, y_pred, label=""):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print(f"{label}MAE:  {mae:.4f}")
    print(f"{label}MSE:  {mse:.4f}")
    print(f"{label}RMSE: {rmse:.4f}")
    print(f"{label}R²:   {r2:.4f}")

def plot_learning_curve(estimator, X, y, title="Curva de aprendizaje"):
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=5, scoring="neg_mean_squared_error", n_jobs=None,
        train_sizes=np.linspace(0.1, 1.0, 5), shuffle=True, random_state=42
    )
    train_rmse = np.sqrt(-train_scores)
    test_rmse = np.sqrt(-test_scores)
    plt.figure()
    plt.title(title)
    plt.xlabel("Tamaño de entrenamiento")
    plt.ylabel("RMSE (CV)")
    plt.plot(train_sizes, train_rmse.mean(axis=1), marker='o', label="Entrenamiento")
    plt.plot(train_sizes, test_rmse.mean(axis=1), marker='s', label="Validación CV")
    plt.legend()
    plt.show()


## Carga y limpieza de datos

In [ ]:
data = load_diabetes()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")
print("Forma de X:", X.shape)
print("Faltantes en X:\n", X.isna().sum())
print("Faltantes en y:", y.isna().sum())

X_capped = cap_outliers_iqr(X, X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_capped, y, test_size=0.2, random_state=42)

num_features = list(X_train.columns)
preprocess = ColumnTransformer(
    transformers=[("num", SimpleImputer(strategy="median"), num_features)]
)


## Modelo base

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ("prep", preprocess),
    ("model", DecisionTreeRegressor(random_state=42))
])

pipe.fit(X_train, y_train)
pred_train = pipe.predict(X_train)
pred_test = pipe.predict(X_test)

print("Rendimiento base:")
report_metrics(y_train, pred_train, label="Train ")
report_metrics(y_test, pred_test, label="Test  ")


## Curva de aprendizaje

In [ ]:
plot_learning_curve(DecisionTreeRegressor(random_state=42), X_train, y_train, title="Curva de aprendizaje — Árbol base (Diabetes)")


## Tuning con GridSearchCV

In [ ]:
param_grid = {
    "model__max_depth": [None, 3, 5, 10],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 5],
    "model__ccp_alpha": [0.0, 0.001, 0.01]
}
grid = GridSearchCV(pipe, param_grid, cv=5, scoring="neg_mean_squared_error")
grid.fit(X_train, y_train)
print("Mejores parámetros:", grid.best_params_)

best_model = grid.best_estimator_
pred_train_best = best_model.predict(X_train)
pred_test_best = best_model.predict(X_test)
print("\nRendimiento con tuning:")
report_metrics(y_train, pred_train_best, label="Train ")
report_metrics(y_test, pred_test_best, label="Test  ")


## Importancias y árbol truncado

In [ ]:
final_tree = best_model.named_steps["model"]
importances = final_tree.feature_importances_
plt.figure()
order = np.argsort(importances)[::-1]
plt.bar(range(len(importances)), importances[order])
plt.xticks(range(len(importances)), np.array(num_features)[order], rotation=45, ha='right')
plt.title("Importancia de características — Diabetes")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plot_tree(final_tree, max_depth=3, filled=True, feature_names=num_features)
plt.title("Árbol de decisión (truncado)")
plt.show()


## Conclusiones
- El tuning suele mejorar la generalización.
- El capado de outliers e imputación hacen el flujo más robusto.
- Las importancias y la visualización del árbol ayudan a interpretar.


## 🧠 Interpretación automática del modelo
Esta celda resume el desempeño del modelo con un texto interpretativo y detecta posibles signos de sobreajuste o subajuste.


In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

# Aseguramos predicciones con el mejor modelo
y_pred_train_best = best_model.predict(X_train)
y_pred_test_best  = best_model.predict(X_test)

# Métricas
mae_tr = mean_absolute_error(y_train, y_pred_train_best)
mse_tr = mean_squared_error(y_train, y_pred_train_best)
rmse_tr = math.sqrt(mse_tr)
r2_tr = r2_score(y_train, y_pred_train_best)

mae_te = mean_absolute_error(y_test, y_pred_test_best)
mse_te = mean_squared_error(y_test, y_pred_test_best)
rmse_te = math.sqrt(mse_te)
r2_te = r2_score(y_test, y_pred_test_best)

print("Resumen de métricas (mejor modelo):")
print(f"Train -> MAE: {mae_tr:.3f} | RMSE: {rmse_tr:.3f} | R²: {r2_tr:.4f}")
print(f"Test  -> MAE: {mae_te:.3f} | RMSE: {rmse_te:.3f} | R²: {r2_te:.4f}\n")

# Interpretación en texto
def nivel(valor, low, mid, high):
    # Helper para categorizar R2
    if valor < low: return "bajo"
    if valor < mid: return "medio"
    if valor < high: return "bueno"
    return "muy bueno"

gap = r2_tr - r2_te
nivel_test = nivel(r2_te, 0.2, 0.5, 0.7)

print("🧠 Interpretación:")
print(f"- El modelo explica aproximadamente el {r2_te*100:.1f}% de la variabilidad en el conjunto de prueba (R² {nivel_test}).")
print(f"- El error promedio absoluto (MAE) en prueba es {mae_te:.2f}, y el RMSE es {rmse_te:.2f}.")
if gap > 0.15:
    print(f"- Existe indicio de **sobreajuste** (diferencia Train-Test de R² = {gap:.2f}). Considera aumentar regularización (max_depth menor, mayor min_samples_leaf/split, o ccp_alpha), o más datos.")
elif gap < -0.05:
    print(f"- El desempeño en prueba es **mejor** que en entrenamiento (gap R² = {gap:.2f}); podría indicar que el train es más ruidoso o que hubo casualidad estadística. Revisa la validación cruzada.")
else:
    print(f"- La generalización es **razonable** (gap R² = {gap:.2f}).")
print("- Si necesitas mayor robustez, compara con RandomForestRegressor o GradientBoostingRegressor.")


## 📊 Comparativa de métricas — Antes vs. Después del *tuning*
La siguiente tabla resume **MAE**, **RMSE** y **R²** para el modelo base y el mejor modelo tras *GridSearchCV*, en **train** y **test**.
